# Проект по SQL

## Описание проекта

Компания купила крупный сервис для чтения книг по подписке. Задача как аналитика — проанализировать базу данных, решить поставленные задачи.
В ней есть информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

## Изучение данных

In [1]:
#импорт библиотек
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
#устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

#сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
#вывод датасетов
for table in ['books', 'authors', 'ratings', 'reviews', 'publishers']:
    query = '''SELECT * FROM {} LIMIT 5'''.format(table)
    data = pd.io.sql.read_sql(query, con = engine)
    display(data)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
#вывод общей информации о датасетах
for table in ['books', 'authors', 'ratings', 'reviews', 'publishers']:
    query = '''SELECT * FROM {} LIMIT 5'''.format(table)
    data = pd.io.sql.read_sql(query, con = engine)
    display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           5 non-null      int64 
 1   author_id         5 non-null      int64 
 2   title             5 non-null      object
 3   num_pages         5 non-null      int64 
 4   publication_date  5 non-null      object
 5   publisher_id      5 non-null      int64 
dtypes: int64(4), object(2)
memory usage: 368.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  5 non-null      int64 
 1   author     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   rating     5 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 288.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   text       5 non-null      object
dtypes: int64(2), object(2)
memory usage: 288.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  5 non-null      int64 
 1   publisher     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


None

Вывод: импортированы библиотеки, просмотрены датасеты и выведена общая информация о датасетах.

## Решение заданий

### Задание 1. Расчет количества книг, вышедших после 1 января 2000 года. 

In [5]:
#количество книг. вышедших после 1 января 2000 года
query = '''
SELECT COUNT(book_id) 
FROM books 
WHERE publication_date > '2000-01-01' 
'''
count_books = pd.io.sql.read_sql(query, engine)

count_books

,count
0,819


Вывод: количество книг, вышедших после 1 января 2000 года, равно 819.

### Задание 2. Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [6]:
#расчет количества обзоров и средней оценки для каждой книги
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT rev.review_id), 
       AVG(rat.rating)
FROM books as b
LEFT JOIN reviews AS rev ON rev.book_id = b.book_id
LEFT JOIN ratings AS rat ON rat.book_id = b.book_id
GROUP BY b.book_id 
ORDER BY AVG(rat.rating)
'''
rev_books = pd.io.sql.read_sql(query, engine)

rev_books

,book_id,title,count,avg
0,303,Harvesting the Heart,2,1.50
1,371,Junky,2,2.00
2,316,His Excellency: George Washington,2,2.00
3,202,Drowning Ruth,3,2.00
4,915,The World Is Flat: A Brief History of the Twen...,3,2.25
...,...,...,...,...
995,421,Marvel 1602,2,5.00
996,418,March,2,5.00
997,169,Crucial Conversations: Tools for Talking When ...,2,5.00
998,901,The Walking Dead Book One (The Walking Dead #...,2,5.00


### Задание 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [7]:
#вывод издательства, которое выпустило наибольшее число книг толще 50 страниц 
query = '''
SELECT p.publisher,
       COUNT(b.book_id)
FROM books AS b
INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50 
GROUP BY p.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1;
'''
publisher = pd.io.sql.read_sql(query, engine)

publisher

,publisher,count
0,Penguin Books,42


### Задание 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками. 

In [8]:
#определение автора с самой высокой средней оценкой книг
query = '''
SELECT tab_1.author,
       AVG(tab_1.avg)

FROM

(SELECT a.author,
       AVG(rat.rating) AS avg, 
       COUNT(rat.rating) AS count 
FROM authors AS a
INNER JOIN books AS b ON a.author_id = b.author_id
INNER JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY a.author, b.book_id
HAVING COUNT(rat.rating) >= 50) AS tab_1
GROUP BY author
ORDER BY AVG(tab_1.avg) DESC
LIMIT 1;

'''
author = pd.io.sql.read_sql(query, engine)

author

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


### Задание 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [9]:
#расчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок
query = '''
WITH tab_1 AS 
(SELECT rev.username, 
       COUNT(review_id) AS count
FROM reviews AS rev
WHERE rev.username in
       (SELECT username
        FROM ratings
        GROUP BY username
        HAVING count(rating_id) > 48)
   GROUP BY username)
SELECT SUM (COUNT) / COUNT(COUNT) AS avg
FROM tab_1
'''
reviews = pd.io.sql.read_sql(query, engine)

reviews

,avg
0,24.0


## Общий вывод по проделанной работе

В ходе проделанной работы:
1. Импортированы необходимые библиотеки;
2. Выведены таблицы из базы данных;
3. Выведена общая информация о таблицах;
4. Решены поставленные задачи. Выявлено:
- количество книг, вышедших после 1 января 2000 года, равно 819;
- для каждой книги посчитаны количество обзоров и средняя оценка: самая наименьшая оценка поставлена книге Harvesting the Heart;
- издательство Penguin Books выпустило 42 книги толще 50 страниц;
- автор J.K. Rowling/Mary GrandPré является автором с самой высокой средней оценкой 4.28;
- среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24. 